# Aphrodisias analysis notebook


In [ ]:
from pathlib import Path

import pygeoprocessing
from osgeo import gdal
import statistics
import math
import numpy as np
import pandas as pd
import rasterio as rio
from rasterio.windows import from_bounds
from shapely.geometry import Polygon 
import rasterio
import itertools
import os
import configparser

### Defining input datasets

In [12]:
gis_folder = Path(r"/Users/cnootenboom/Documents/Projects/Circuitscape")

target_nodata = -9999.0

dem_raster_path = gis_folder / "DEM_UTM_35N.tif"
dem_raster_info = pygeoprocessing.get_raster_info(str(dem_raster_path))

slope_raster_path = gis_folder / "slope_35N.tif"
tri_raster_path = gis_folder / "tri_35N.tif"
landcover_raster_path = gis_folder / "landcover_35N.tif"

church_vector_path = gis_folder / "churches_35N.gpkg"
city_center_vector_path = gis_folder / "center_35N.gpkg"

church_raster_path = gis_folder / "churches_35N.tif"
city_center_raster_path = gis_folder / "center_35N.tif"

### Rasterizing Churches to match the DEM

In [3]:
# Create and burn church locations
if not church_raster_path.exists:
    pygeoprocessing.new_raster_from_base(
        str(dem_raster_path),
        str(church_raster_path),
        gdal.GDT_Byte,
        [0],
    )
    pygeoprocessing.rasterize(
    str(church_vector_path),
    str(church_raster_path),
    [1],
    )

### Rasterizing city center to match the DEM

In [ ]:
# Create and burn city center
if not city_center_raster_path.exists:
    pygeoprocessing.new_raster_from_base(
        str(dem_raster_path),
        str(city_center_raster_path),
        gdal.GDT_Byte,
        [0],
    )
    pygeoprocessing.rasterize(
    str(city_center_vector_path),
    str(city_center_raster_path),
        [1],
    )

### Calculate slope and Terrain Ruggedness Index from DEM

In [ ]:
slope_gdal_ds = gdal.DEMProcessing(str(slope_raster_path), str(dem_raster_path), "slope")
del slope_gdal_ds

# pygeoprocessing.calculate_slope((str(dem_raster_path), 1), str(slope_raster_path))

tri_gdal_ds = gdal.DEMProcessing(str(tri_raster_path), str(dem_raster_path), "tri")
del tri_gdal_ds

### Normalize input datasets

In [ ]:
#Align and clip landscape data to DEM extent
dem_raster_info = pygeoprocessing.get_raster_info(str(dem_raster_path))
dem_raster_clip = gis_folder / "DEM_UTM_35N_clip.tif"
landcover_raster_clip = gis_folder / "landcover_35N_clip.tif"
slope_raster_clip = gis_folder / "slope_35N_clip.tif"
tri_raster_clip = gis_folder / "tri_35N_clip.tif"

pygeoprocessing.align_and_resize_raster_stack(
    [str(dem_raster_path), str(landcover_raster_path), str(slope_raster_path), str(tri_raster_path)],
    [str(dem_raster_clip), str(landcover_raster_clip), str(slope_raster_clip), str(tri_raster_clip)],
    ["near", "near", "near", "near"],
    dem_raster_info["pixel_size"],
    dem_raster_info["bounding_box"],
    raster_align_index=0,
)

### Reclassify landcover into cost surface

In [ ]:
reclass_df = pd.read_csv("./data/esa_worldcover_classification.csv")
cost_value = "cost_value"
landcover_reclass = gis_folder / "landcover_reclass.tif"

reclass_dict = reclass_df.set_index("lucode").to_dict()[cost_value]

pygeoprocessing.reclassify_raster(
    (str(landcover_raster_clip),1),
    reclass_dict,
    str(landcover_reclass),
    gdal.GDT_Float32,
    target_nodata,
)

### Calculate Tobler's hiking function

In [19]:
tobler_surface_raster = gis_folder / "tobler_surface.tif"

slope_raster_info = pygeoprocessing.get_raster_info(str(slope_raster_clip))
slope_cell_resolution = statistics.mean([abs(x) for x in slope_raster_info["pixel_size"]])
slope_nodata = slope_raster_info["nodata"][0]

# def tobler_op(slope_array):
#     result = (slope_cell_resolution/1000)/(6*np.exp(-3.5*np.abs(np.tan(slope_array*math.pi/180)+0.05)))
#     return result
# def tobler_op(slope):
#     result = (slope_cell_resolution/1000)/(6*np.exp(-3.5*np.abs(slope+0.05)))
#     return result

def tobler_op(slope_array):
    # Make an array of the same shape full of nodata
    output = np.full(slope_array.shape, slope_nodata)

    # Make a masking array to ignore all nodata areas in the original data
    valid_mask = np.full(slope_array.shape, True)
    valid_mask &= ~pygeoprocessing.array_equals_nodata(slope_array, slope_nodata)

    output[valid_mask] = (slope_cell_resolution/1000)/(6*np.exp(-3.5*np.abs(np.tan(slope_array[valid_mask]*math.pi/180)+0.05)))

    return output

if not tobler_surface_raster.exists():
    pygeoprocessing.raster_calculator(
        [(str(slope_raster_clip),1)], 
        tobler_op,
        str(tobler_surface_raster),
        gdal.GDT_Float32,
        target_nodata,
        calc_raster_stats=True
    )


# Define Tobler rescaling function
def tobler_rescale_op(tobler_surface_array, upper_limit=0.1666667, lower_limit=0.0):
    # Make an array of the same shape full of nodata
    output = np.full(tobler_surface_array.shape, target_nodata)

    # Make a masking array to ignore all nodata areas in the original data
    valid_mask = np.full(tobler_surface_array.shape, True)
    valid_mask &= ~pygeoprocessing.array_equals_nodata(tobler_surface_array, target_nodata)

    # Calculate initial rescaling
    output[valid_mask] = (tobler_surface_array[valid_mask] - lower_limit)/(upper_limit - lower_limit)

    # Force values larger than the upper_limit to equal one
    upper_limit_mask = (tobler_surface_array >= upper_limit) & valid_mask
    output[upper_limit_mask] = 1

    # Force values smaller than the lower_limit to equal zero
    lower_limit_mask = (tobler_surface_array <= lower_limit) & valid_mask
    output[lower_limit_mask] = 0

    return output

tobler_rescale_raster = gis_folder / "tobler_rescale.tif"

# Rescaling Tobler's original function
if tobler_rescale_raster.exists():
    tobler_rescale_raster.unlink()
pygeoprocessing.raster_calculator(
    [(str(tobler_surface_raster),1)], 
    tobler_rescale_op,
    str(tobler_rescale_raster),
    gdal.GDT_Float32,
    target_nodata,
)

### Create multicriteria cost surface

In [20]:
cost_surface = gis_folder / "cost_surface.tif"

# Get nodata values for toblers and lulc
landcover_raster_nodata = pygeoprocessing.get_raster_info(str(landcover_reclass))["nodata"][0]
tobler_raster_nodata = pygeoprocessing.get_raster_info(str(tobler_rescale_raster))["nodata"][0]


def weighted_average_op(tobler_array, lulc_cost_array, tobler_weight=0.8, lulc_cost_weight=0.2):
    # Make an array of the same shape full of nodata
    output = np.full(tobler_array.shape, target_nodata)

    # Make a masking array to ignore all nodata areas in the original data
    valid_mask = np.full(tobler_array.shape, True)
    valid_mask &= ~pygeoprocessing.array_equals_nodata(tobler_array, target_nodata)
    valid_mask &= ~pygeoprocessing.array_equals_nodata(lulc_cost_array, target_nodata)

    # Calculate weighted average
    output[valid_mask] = ((tobler_array[valid_mask]*tobler_weight) + (lulc_cost_array[valid_mask]*lulc_cost_weight)) / (tobler_weight+lulc_cost_weight)

    return output 

if cost_surface.exists():
    cost_surface.unlink()
pygeoprocessing.raster_calculator(
    [(str(tobler_rescale_raster),1),(str(landcover_reclass),1)],
    weighted_average_op,
    str(cost_surface),
    gdal.GDT_Float32,
    target_nodata,
)

### Calculate combined friction surface using raster calculator

In [ ]:
friction_surface = gis_folder / "friction_surface.tif"

file_list = [
    str(friction_surface),
    str(friction_surface_1),
    str(friction_surface_2)
]

def friction_op(slope, vegetation):
    result = slope*0.8 + vegetation*2
    return result

if not friction_surface.exists():
    pygeoprocessing.raster_calculator(
        file_list,
        friction_op,
        str(friction_surface),
        gdal.GDT_Float32,
        -1,
    )

### Run tiled Circuitscape method

In [ ]:
from .circuitscape import circuitscape


cost_surface_raster = gis_folder / "cost_surface.tif"
workspace_path = gis_folder/ "circuitscape"
seed_ini = gis_folder / "circuitscape/blank_circuitscape_ini.ini"
# seed_ini = gis_folder / "circuitscape/blank_circuitscape_manual.ini"

circuitscape(cost_surface_raster, seed_ini, workspace_path, keep_intermediates=True)
